##### EEEL 4220 Final Project
## Wind Farm Sizing and Siting on IEEE-14 System

4. 14-bus 5-scenario, with wind planning. Finally, assemble all scenarios together to pro-
duce the full planning model.

In [1]:
# Import libraries
import cvxpy as cp
import numpy as np
import pandas as pd

In [2]:
# Load data
b_df = pd.read_csv('buses.csv')
g_df = pd.read_csv('generators.csv')
l_df = pd.read_csv('branches.csv')
d_df = pd.read_csv('demand_scenarios.csv')
cfs_df = pd.read_csv('wind_cf_scenarios.csv')
y_df = pd.read_csv('wind_candidates.csv')

In [3]:
# Parameters
c2 = g_df['c2'].to_numpy() # quadratic cost coefficients for generator g
c1 = g_df['c1'].to_numpy() # linear cost coefficients for generator g
c0 = g_df['c0'].to_numpy() # fixed cost coefficients for generator g
Pmin = g_df['Pmin_MW'].to_numpy() # minimum generation limits for generator g
Pmax = g_df['Pmax_MW'].to_numpy() # maximum generation limits for generator g
x = l_df['x_pu'].to_numpy() # reactance of transmission line l
fmax = l_df['rateA_MW'].to_numpy() # thermal limit of line l
D = d_df.pivot(index='bus_id',columns='scenario_id',values='Pd_MW').to_numpy() # demand at bus b in scenario s
CF = cfs_df['capacity_factor'].to_numpy() # wind capacity factor in scenario s
omega = cfs_df['probability'].to_numpy() # probability (weight) of scenario s
ybar = pd.merge(b_df, y_df, on='bus_id', how='left')['max_capacity_MW'].fillna(0).to_numpy() # maximum installable wind capacity at bus b (0 if not candidate)
Cinv = 1.5 # investment cost per MW of installed wind capacity ($/MW)

# Decision variables
# ps = power output of generator g under scenario s
p1 = cp.Variable(len(g_df), nonneg=True)
p2 = cp.Variable(len(g_df), nonneg=True)
p3 = cp.Variable(len(g_df), nonneg=True)
p4 = cp.Variable(len(g_df), nonneg=True)
p5 = cp.Variable(len(g_df), nonneg=True)
p = [p1, p2, p3, p4, p5]
# fs = power flow on line l under scenario s
f1  = cp.Variable(len(l_df))
f2  = cp.Variable(len(l_df))
f3  = cp.Variable(len(l_df))
f4  = cp.Variable(len(l_df))
f5  = cp.Variable(len(l_df))
f = [f1, f2, f3, f4, f5]
# thetas = voltage angle at bus b (radians)
theta1 = cp.Variable(len(b_df))
theta2 = cp.Variable(len(b_df))
theta3 = cp.Variable(len(b_df))
theta4 = cp.Variable(len(b_df))
theta5 = cp.Variable(len(b_df))
# ws = wind generation at bus b under scenario s
theta = [theta1, theta2, theta3, theta4, theta5]
w1 = cp.Variable(len(b_df), nonneg=True)
w2 = cp.Variable(len(b_df), nonneg=True)
w3 = cp.Variable(len(b_df), nonneg=True)
w4 = cp.Variable(len(b_df), nonneg=True)
w5 = cp.Variable(len(b_df), nonneg=True)
w = [w1, w2, w3, w4, w5]
# installed wind capacity at bus b (MW)
y = cp.Variable(len(b_df), nonneg=True)

In [4]:
# Initialize incidence matrices
# Generator–bus
# Ggb = 1 if generator g is connected to bus b, 0 otherwise.
G = np.zeros((len(g_df), len(b_df)))

# Line–bus
# Alb = 1 if bus b is the sending (“from”) end of line l, if bus b is the receiving (“to”) end of line l, 0 otherwise.
A = np.zeros((len(l_df), len(b_df)))

# Populate incidence matrices
# Generator–bus
for gen_index in range(len(g_df)):
    bus_id = g_df.loc[gen_index, 'bus_id']
    bus_index = bus_id - 1
    G[gen_index, bus_index] = 1

# Line–bus
for line_index in range(len(l_df)):
    from_bus_id = l_df.loc[line_index, 'from_bus']
    to_bus_id = l_df.loc[line_index, 'to_bus']
    from_bus_index = from_bus_id - 1
    to_bus_index = to_bus_id - 1
    A[line_index, from_bus_index] = 1
    A[line_index, to_bus_index] = -1

In [ ]:
# Loop through scenarios
ls = len(cfs_df)
# Initialize results
CT = np.zeros(shape=(1, len(cfs_df)))
MCT = np.zeros(shape=((len(b_df),len(cfs_df))))
GT = np.zeros(shape=((len(g_df),len(cfs_df)))) 
WT = np.zeros(shape=((len(b_df),len(cfs_df))))
LT = np.zeros(shape=((len(l_df),len(cfs_df))))

CT = pd.DataFrame(CT, index = ['Total_Cost'], columns=['S1', 'S2', 'S3', 'S4', 'S5'])
MCT = pd.DataFrame(MCT, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'],
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
GT = pd.DataFrame(GT, index = ['G1', 'G2', 'G3', 'G4', 'G5'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
WT = pd.DataFrame(WT, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
LT = pd.DataFrame(LT, index = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])

for s in range(ls):
    # Define constraints
    # Initialize an empty constraint set
    con = [] 

    # power balance
    con.append((A.T @ f[s]) + D[:, s] == (G.T @ p[s]) + w[s])

    # generator output limits
    con.append(p[s] <= Pmax)  # maximum generation

    # set reference angle
    con.append(theta[s][0] == 0)

    # DC power flow
    con.append(f[s] == (A @ theta[s]) / x)

    # line flow limits
    con.append(f[s] <= fmax)  # maximum line flow
    con.append(f[s] >= -fmax)  # minimum line flow

    # wind generation limits
    con.append(w[s] <= y * CF[s])  # wind generation cannot exceed installed capacity
    
    # wind capacity limits
    con.append(y <= ybar)  # maximum installed capacity

    # Define objective function
    obj = cp.Minimize(cp.sum(cp.multiply(c2, cp.square(p[s])) + cp.multiply(c1, p[s]) + c0) 
                      + Cinv * cp.sum(y))
    
    # Solve problem
    prob = cp.Problem(obj, con)
    prob.solve(solver = "Gurobi");

    # Store results
    CT.iloc[0, s] = obj.value
    MCT.iloc[:, s] = con[0].dual_value
    GT.iloc[:, s] = p[s].value
    WT.iloc[:, s] = w[s].value
    LT.iloc[:, s] = f[s].value


In [8]:
w[4].value

array([0.00000000e+00, 1.74793221e-10, 0.00000000e+00, 0.00000000e+00,
       1.67097847e-10, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.02209209e-10, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.52115531e-10, 0.00000000e+00])

In [ ]:
# Initialize results
CT = np.zeros(shape=(1, len(cfs_df)))
MCT = np.zeros(shape=((len(b_df),len(cfs_df))))
GT = np.zeros(shape=((len(g_df),len(cfs_df)))) 
WT = np.zeros(shape=((len(b_df),len(cfs_df))))
LT = np.zeros(shape=((len(l_df),len(cfs_df))))
CT = pd.DataFrame(CT, index = ['Total_Cost'], columns=['S1', 'S2', 'S3', 'S4', 'S5'])
MCT = pd.DataFrame(MCT, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'],
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
GT = pd.DataFrame(GT, index = ['G1', 'G2', 'G3', 'G4', 'G5'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
WT = pd.DataFrame(WT, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])
LT = pd.DataFrame(LT, index = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20'], 
                  columns=['S1', 'S2', 'S3', 'S4', 'S5'])

In [ ]:
# Print results
print("Total generation in the system for each scenario:")
print((GT.sum(axis=0).round(1) + WT.sum(axis=0).round(1)).to_list())
print("\n Total demand in the system for each scenario:") 
print(D.sum(axis=0).round(1))


    # Store results
    CT.iloc[0, s] = obj.value
    MCT.iloc[:, s] = con[0].dual_value
    GT.iloc[:, s] = p[s].value
    WT.iloc[:, s] = w[s].value
    LT.iloc[:, s] = f[s].value

print("\n Total operating cost:")
print(CT.round(1))

print("\n Expected operating cost:")
print((omega @ CT.values[0,:]).round(1))

print("\n Thermal generator dispatch results: ")
print(GT.round(1))

print("\n Wind dispatch results: ")
print(WT.round(1))

print("\n Line flow results: ")
print(LT.round(1))

print("\n Locational Marginal Prices: ")
print(MCT.round(1))

Total generation in the system for each scenario:
[233.1, 259.0, 284.9, 246.0, 271.9]

 Total demand in the system for each scenario:
[233.1 259.  284.9 246.  271.9]

 Total operating cost:
               S1     S2     S3     S4     S5
Total_Cost  721.2  831.8  789.4  778.7  656.0

 Expected operating cost:
755.4

 Thermal generator dispatch results: 
      S1    S2    S3    S4    S5
G1  79.1  76.2  33.3  83.8  16.7
G2  42.9  41.4  20.0  45.2  11.7
G3  43.5  41.7  16.0  46.3   6.0
G4  35.9  34.8  18.8  37.7  12.5
G5  31.7  30.9  18.0  33.1  13.0

 Wind dispatch results: 
      S1    S2    S3   S4    S5
B1   0.0   0.0   0.0  0.0   0.0
B2   0.0   6.9  46.7  0.0  61.6
B3   0.0   0.0   0.0  0.0   0.0
B4   0.0   0.0   0.0  0.0   0.0
B5   0.0   6.1  30.7  0.0  36.7
B6   0.0   0.0   0.0  0.0   0.0
B7   0.0   0.0   0.0  0.0   0.0
B8   0.0   0.0   0.0  0.0   0.0
B9   0.0  14.3  69.2  0.0  82.1
B10  0.0   0.0   0.0  0.0   0.0
B11  0.0   0.0   0.0  0.0   0.0
B12  0.0   0.0   0.0  0.0   0.0
B13  0